# Test for inversecovariance benchmarking against CorrCal-C implimentation

In [1]:
import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.insert(0, str(parent_dir))

In [2]:
import numpy as np
import cupy as cp
import seaborn as sns
from corrcal import sparse
from zp_puregpu_funcs_py import *
from invcov import *

hera_cal is not installed. Some gridding features unavailable.
pyfof is not installed. Some gridding features unavailable.


In [3]:
#the main parametes describing our problem.
n_bl = 120000
n_eig = 3
n_src = 5
xp = np  #run things on the gpu using cupy

#random array of edges for the diffuse matrix
edges = xp.unique(xp.random.randint(1, n_bl-1, size = 500))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
# print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
# sim_noise_mat = xp.random.rand(n_bl, dtype = 'float64') + 0.1   #in principle this is squared since is a variance
# sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
# sim_src_mat = xp.random.rand(n_bl, n_src, dtype = 'float64')
sim_noise_mat = xp.random.rand(n_bl) + 0.1   #in principle this is squared since is a variance
sim_diff_mat = xp.random.rand(n_bl, n_eig)
sim_src_mat = xp.random.rand(n_bl, n_src)
# sim_noise_mat_np = cp.asnumpy(sim_noise_mat)
# sim_diff_mat_np = cp.asnumpy(sim_diff_mat)
# sim_src_mat_np = cp.asnumpy(sim_src_mat)
# edges_np = cp.asnumpy(edges)
# src_mat = sim_src_mat_np @ sim_src_mat_np.T.conj()  # Ensures src_mat is positive definite
# diff_mat = sim_diff_mat_np @ sim_diff_mat_np.T.conj()
# noise_mat = sim_noise_mat_np @ sim_noise_mat_np.T.conj()

In [11]:
print(sim_diff_mat.dtype)

float64


In [4]:
cov_instance = sparse.SparseCov(sim_noise_mat, sim_src_mat, sim_diff_mat, edges=edges, n_eig=n_eig)
# cov_instance = sparse.SparseCov(noise_mat, src_mat, diff_mat, edges=edges_np, n_eig=n_eig)

In [5]:
cov_inv, log_det = cov_instance.inv(return_det=True)

In [6]:
test_results = str(benchmark(cov_instance.inv, n_repeat=100))
test_results = test_results.split()
cpu_t = float(test_results[3])/1e6
gpu_t = float(test_results[14])/1e6
print(f"Time on cpu: {cpu_t:.6f}s")
print(f"Time on gpu: {gpu_t:.6f}s")

Time on cpu: 0.101809s
Time on gpu: 0.102384s


In [7]:
#the main parametes describing our problem.
n_bl = 120000
n_eig = 3
n_src = 5
xp = cp  #run things on the gpu using cupy

#random array of edges for the diffuse matrix
edges = xp.unique(xp.random.randint(1, n_bl-1, size = 500))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
# print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
sim_noise_mat = xp.random.rand(n_bl, dtype = 'float64') + 0.1   #in principle this is squared since is a variance
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_src_mat = xp.random.rand(n_bl, n_src, dtype = 'float64')


In [8]:
#zeropad the noise, diff, source mats
zp_sim_noise_mat_inv, largest_block, n_blocks = zeroPad(sim_noise_mat, edges, return_inv=True)  #return inverse set to True to get around div by 0 issue
zp_sim_noise_mat, largest_block, n_blocks = zeroPad(sim_noise_mat, edges, return_inv=False)  #return inverse set to False for sims later on
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)
zp_sim_src_mat, largest_block, n_blocks = zeroPad(sim_src_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_noise_mat_2d = zp_sim_noise_mat_inv.reshape(n_blocks, largest_block)   #Again, its worth noting the values of the noise mat are really 1/noise_vals here
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)
sim_src_mat_3d = zp_sim_src_mat.reshape(n_blocks, largest_block, n_src)

In [9]:
test_results = str(benchmark(inverse_covariance, (sim_noise_mat_2d, sim_diff_mat_3d, sim_src_mat_3d, edges, xp, True, True), n_repeat=100))
test_results = test_results.split()
cpu_t = float(test_results[3])/1e6
gpu_t = float(test_results[14])/1e6
print(f"Time on cpu: {cpu_t:.6f}s")
print(f"Time on gpu: {gpu_t:.6f}s")

Time on cpu: 0.033856s
Time on gpu: 0.034011s
